## Web Assembly

In [1]:
#include <base/net/HTTPSRequest.h>
#include <base/string/Format.h>
#include <base/string/FormatOutputStream.h>

In [2]:
using namespace base;

In [3]:
String url = "https://github.com/rene-fonseca/docker-webassembly/raw/master/wasmtime/hello";

In [4]:
auto response = HTTPSRequest::load(url)

In [5]:
response.getSecond()

"application/octet-stream"

In [6]:
response.getFirst().getLength()

31259

In [7]:
#include <base/webassembly/WebAssembly.h>

In [8]:
WebAssembly::isSupported()

true

In [9]:
WebAssembly wasm;

In [10]:
wasm.getEngine()

"Wasmtime 0.8"

In [11]:
wasm.isValid(response.getFirst())

true

In [12]:
wasm.load(response.getFirst())

true

In [13]:
fout << "IMPORTS:" << EOL;
for (const auto& s : wasm.getImports()) {
  fout << "  " << "[" << s.index << "] ";
  switch (s.externType) {
  case WebAssembly::EXTERN_FUNCTION:
    fout << "FUNCTION " << WebAssembly::toString(s, true) << ENDL;
    break;
  case WebAssembly::EXTERN_GLOBAL:
    fout << "GLOBAL " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_TABLE:
    fout << "TABLE " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_MEMORY:
    fout << "MEMORY " << s.name << ENDL;
    break;
  default:
    fout << "?" << " " << s.name << ENDL;
  }
}

IMPORTS:
  [0] FUNCTION [i32] wasi_unstable!fd_prestat_get(i32, i32)
  [1] FUNCTION [i32] wasi_unstable!fd_prestat_dir_name(i32, i32, i32)
  [2] FUNCTION [] wasi_unstable!proc_exit(i32)
  [3] FUNCTION [i32] wasi_unstable!fd_fdstat_get(i32, i32)
  [4] FUNCTION [i32] wasi_unstable!fd_close(i32)
  [5] FUNCTION [i32] wasi_unstable!args_sizes_get(i32, i32)
  [6] FUNCTION [i32] wasi_unstable!args_get(i32, i32)
  [7] FUNCTION [i32] wasi_unstable!fd_write(i32, i32, i32, i32)
  [8] FUNCTION [i32] wasi_unstable!fd_seek(i32, i64, i32, i32)


In [14]:
wasm.makeInstance(true)

true

In [15]:
fout << "EXPORTS:" << EOL;
for (const auto& s : wasm.getExports()) {
  fout << "  " << "[" << s.index << "] ";
  switch (s.externType) {
  case WebAssembly::EXTERN_FUNCTION:
    fout << "FUNCTION " << WebAssembly::toString(s, true) << ENDL;
    break;
  case WebAssembly::EXTERN_GLOBAL:
    fout << "GLOBAL " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_TABLE:
    fout << "TABLE " << s.name << ENDL;
    break;
  case WebAssembly::EXTERN_MEMORY:
    fout << "MEMORY SIZE=" << s.memorySize << ENDL;
    break;
  default:
    fout << "?" << " " << s.name << ENDL;
  }
}

EXPORTS:
  [0] MEMORY SIZE=131072
  [1] FUNCTION [] _start()


In [16]:
wasm.call("_start")

WebAssemble: calling: _start()
wasi_unstable!fd_prestat_get(i32: 3 = 0x3, i32: 70808 = 0x11498) -> (i32: 0 = 0x0) INVOKES=1


Error: 

In [17]:
const char* wat = R"WAT((module
    ;; Import the required fd_write WASI function which will write the given io vectors to stdout
    ;; The function signature for fd_write is:
    ;; (File Descriptor, *iovs, iovs_len, nwritten) -> Returns number of bytes written
    (import "wasi_unstable" "fd_write" (func $fd_write (param i32 i32 i32 i32) (result i32)))

    (memory 1)
    (export "memory" (memory 0))

    ;; Write 'hello world\n' to memory at an offset of 8 bytes
    ;; Note the trailing newline which is required for the text to appear
    (data (i32.const 8) "hello world\n")

    (func $main (export "_start")
        ;; Creating a new io vector within linear memory
        (i32.store (i32.const 0) (i32.const 8))  ;; iov.iov_base - This is a pointer to the start of the 'hello world\n' string
        (i32.store (i32.const 4) (i32.const 12))  ;; iov.iov_len - The length of the 'hello world\n' string

        (call $fd_write
            (i32.const 1) ;; file_descriptor - 1 for stdout
            (i32.const 0) ;; *iovs - The pointer to the iov array, which is stored at memory location 0
            (i32.const 1) ;; iovs_len - We're printing 1 string stored in an iov - so one.
            (i32.const 20) ;; nwritten - A place in memory to store the number of bytes written
        )
        drop ;; Discard the number of bytes written from the top of the stack
    )
))WAT";

In [18]:
WebAssembly wasm2;

Text format (WAT) is not yet supported.

In [19]:
wasm2.isValid(wat)

false

In [20]:
wasm2.load(wat)

false